In [50]:
from clean_data import clean_data
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import pandas as pd
import numpy as np
import os 

import geopandas as gpd
from matplotlib import pyplot as plt
from matplotlib import colors as c
import matplotlib.ticker as ticker


plt.rcParams["figure.figsize"] = (16,16)
plt.rcParams["figure.dpi"] = 440

In [27]:
# This is a constant that will be used for anything
# that needs uses randomness, so that our tests stay
# consistent and repeatable
RANDOM_STATE = 0
METRICS = ['accuracy', 'f1_macro', 'f1_micro']
NUM_FEAT_IMPORTANCES = 10
DOWNSAMPLE_LABEL = 1

### Data Loading and Formatting

In [28]:
def read_data_and_format(filepath: str = "./data/utah_county_accidents.csv", 
                         test_perc: float = .3,
                         index_column = 'ID',
                         label_column = 'Severity',
                         columns_to_drop: list = ['Unnamed: 0', 'End_Time', 
                                                  'County', 'State', 'City', 
                                                  'Country', 'Timezone', 
                                                  'Airport_Code', 'Street', 
                                                  'Zipcode', 'Source', 
                                                  'Description', 'Weather_Timestamp', 
                                                  'Wind_Direction', 'Nautical_Twilight', 
                                                  'Astronomical_Twilight'],
                         dummy_columns: list = ['Month', 'Day', 'Civil_Twilight', 'Sunrise_Sunset'],
                         rand_state=RANDOM_STATE,
                         stratify=True,
                         ):
    """
    This function reads and formats the data to have the correct columns
    and cleans the columns to have the right features engineered.

    Params
    -------
    filepath (str): the path to the data
    test_perc (float): the percent of test data
    rand_state (bool): Should stay true (Don't change unless the group all agrees)
    stratify (bool): Should be True (that way each class is equally represented
                     in the test and train set)

    Returns
    -------
    X_train (pd.DataFrame): the training data
    y_train (pd.DataFrame): the training labels
    X_test (pd.DataFrame): the testing data
    y_test (pd.DataFrame): the testing labels
    """
    df = pd.read_csv(filepath, index_col=index_column)

    df = clean_data(df, to_drop=columns_to_drop)

    df = pd.get_dummies(df, columns=list(set(dummy_columns) - set(columns_to_drop)))
    df_y = df[label_column].copy()
    df_X = df.drop(columns=[label_column])

    # stratify makes sure each class is equally represented precentage wise in the split
    if stratify:
        df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=test_perc, random_state=rand_state, stratify=df_y)
    elif test_perc == 0:
        df_X_train, df_X_test, df_y_train, df_y_test = df_X, _, df_y, _
    else:
        df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=test_perc, random_state=rand_state)

    # Note that I am shifiting all the labels so it works with all the models
    if not (0 in list(df_y_train.unique())):
        df_y_train = df_y_train - 1
        if test_perc != 0:
            df_y_test = df_y_test - 1

    return df_X_train, df_y_train, df_X_test, df_y_test

In [29]:
def down_sample(X: pd.DataFrame, y: pd.Series, label: int) -> pd.DataFrame:
    """

    This function downsamples all labels to the specified label

    Params
    -------
    X (pd.DataFrame): dataframe of training data
    y (pd.Series): training labels
    label (int): label to downsample to

    Returns
    -------
    X_train (pd.DataFrame): down sampled training data
    y_train (pd.Series): down sampled training labels
    """
    # Find the number of occurances of the downsample label
    counts = y.value_counts()
    num_keep = len(y[y == label])

    # Data copies to edit
    X_copy = X.copy()
    y_copy = y.copy()
    
    # loop through data point of each label type and 
    # drop a random subset so the class type has as
    # no more occurances than the downsample label
    for class_type in counts.index:
        num_drop = counts[class_type] - num_keep
        if num_drop > 0:
            drop_indices = np.random.choice(y_copy[y_copy == class_type].index, num_drop, replace=False)
            X_copy.drop(index=drop_indices, inplace=True)
            y_copy.drop(index=drop_indices, inplace=True)
    
    return X_copy, y_copy

## Utah County Model - OOD Scoring (On Other Utah Counties)

### Prepare data

In [ ]:
# Get Training data for the Utah County Model
file = 'utah_county_accidents.csv'
filepath = './data/' + file
X_train, y_train, X_test, y_test = read_data_and_format(filepath)
X_train, y_train = down_sample(X_train, y_train, DOWNSAMPLE_LABEL)

In [ ]:
########
# Get all the Utah counties formatted and split up
########
file = 'utah_accidents.csv'
filepath = './data/' + file
counties_X_test, counties_y_test, _, _ = read_data_and_format(filepath, test_perc=0, columns_to_drop=['Unnamed: 0', 'End_Time', 
                                                                                    'State', 'City', 
                                                                                    'Country', 'Timezone', 
                                                                                    'Airport_Code', 'Street', 
                                                                                    'Zipcode', 'Source', 
                                                                                    'Description', 'Weather_Timestamp', 
                                                                                    'Wind_Direction', 'Nautical_Twilight', 
                                                                                    'Astronomical_Twilight'],
                                                                                    stratify=False)
counties_in_utah = counties_X_test.County.unique()

# Create a test_set for each county dataset
county_testsets = []
for county in counties_in_utah:
    if county == 'Utah':
        county_testsets.append((county, X_test, y_test))
    else:
        test_X = counties_X_test[counties_X_test.County == county] 
        test_X.drop(columns=['County'], inplace=True)
        test_y = counties_y_test[counties_X_test.County == county]

        county_testsets.append((county, test_X, test_y))

### Get the Best Utah County Model

In [36]:
# Params for the best Utah County model
params = {'class_weight': 'balanced_subsample', 
          'criterion': 'entropy', 
          'max_features': 42, 
          'min_samples_leaf': 3, 
          'n_estimators': 50}

# Make the trees
model = RandomForestClassifier(random_state=RANDOM_STATE, 
                               **params, warm_start=False)

# Fit the trees
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_features=42, min_samples_leaf=3, n_estimators=50,
                       random_state=0)

### Run OOD Tests on Utah Counties

In [42]:
# Loop through each county data subset and run predict
# severity on the crash type
uc_f1score_dict = {}
for county, X, y in county_testsets:
    y_pred = model.predict(X)
    score = f1_score(y, y_pred, average='macro', zero_division=1)
    uc_f1score_dict[county] = score

# Create a pd.DF of the scores and find the average
ood_uc = pd.Series(uc_f1score_dict)
print('Average OOD score for Utah Counties (excluding Utah County): ', ood_uc[ood_uc.index != 'Utah'].mean())

Average OOD score for Utah Counties (excluding Utah County):  0.32606820952394794


### Plot OOD Scores

In [46]:
# Get the counties data formated correctly
counties = gpd.read_file("./data/county_data.gpkg")
utah = counties[counties['STATEFP'] == '49']
utah = utah.sort_values(by="COUNTYFP")

# read in the ood scores as a dataframe, sort them alphabetically, and get the score values out
ood_scores = pd.read_csv("./data/ood_uc.csv").sort_values(by="county")["rf_score"].values

# add the scores to the utah df
utah['best_score'] = ood_scores

# Make a pretty plot
fig, ax = plt.subplots(1, figsize=(30, 30))
base = utah.boundary.plot(edgecolor="black", ax=ax, linewidth=1)

# plot the colors of the counties by rf_score for that county
utah.plot(column='best_score', ax=base, cmap='viridis_r', legend=True)

# now add the names of the counties to the map as well (help from StackOverflow to get this command)
utah.apply(lambda x: ax.annotate(text=x.NAME + ": {0}".format(str(x.best_score).lstrip('0')[:3]), xy=x.geometry.centroid.coords[0], ha='center', color="black" if x.best_score < .45 else "white", fontsize=12), axis=1)

# finish the plotting
ax.set_xlabel('Longitude', fontsize=15)
ax.set_ylabel('Latitude', fontsize=15)
ax.set_title('Out of Distribution RF Scores by County', fontsize=24)
fig_stuff = plt.gcf()
cax = fig_stuff.get_axes()[1]
cax.set_ylabel("F1-Score", fontsize=15)
plt.show()

## Utah Model - OOD Scoring (On Other US States)

### Prepare data

In [ ]:
# Get the Utah county accident with the right split
file = 'utah_accidents.csv'
filepath = './data/' + file
X_train, y_train, X_test, y_test = read_data_and_format(filepath)
X_train, y_train = down_sample(X_train, y_train, DOWNSAMPLE_LABEL)

### Get the Best Utah Model

In [48]:
# The parameters for the best model found 
# in the grid search
params = {'class_weight': 'balanced', 'criterion': 'entropy', 'max_features': 30, 'min_samples_leaf': 6, 'n_estimators': 150}

# Make the trees
model = RandomForestClassifier(random_state=RANDOM_STATE, **params, warm_start=False)

# Fit the trees
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_features=30, min_samples_leaf=6, n_estimators=150,
                       random_state=0)

### Run OOD Tests on the US 

In [1]:
def evaluate_state_f1score(filename, utah_col_set, utah_cols, utah_modes):
    """
    This function reads in a state accident dataset and formats it. Then
    it runs it predicts accident severity using the Utah Model, and returns
    the corresponding F1-score

    Params
    -------
    filename (str): the name of the data csv
    utah_col_set (set): A set of the utah columns to find missing columns
    utah_cols (list): A list of the utah columns
    utah_modes (list): A list of the Utah datasets column modes 

    Returns
    -------
    f1_score (float): the f1_score from the models prediction on the given
                      state datset
    """
    filepath = './data/states/' + filename
    states_X_test, states_y_test, _, _ = read_data_and_format(filepath, test_perc=0, columns_to_drop=['Unnamed: 0', 'End_Time', 
                                                                                        'State', 'City', 
                                                                                        'Country', 'County', 'Timezone', 
                                                                                        'Airport_Code', 'Street', 
                                                                                        'Zipcode', 'Source', 
                                                                                        'Description', 'Weather_Timestamp', 
                                                                                        'Wind_Direction', 'Nautical_Twilight', 
                                                                                        'Astronomical_Twilight'],
                                                                                        stratify=False)
    
    # Remove any excess columns they have 
    state_col_set = set(states_X_test.columns)
    columns_to_remove = state_col_set - utah_col_set
    if len(columns_to_remove) != 0:
        print("Cols to Remove: ", columns_to_remove)
        states_X_test.drop(columns=list(columns_to_remove), inplace=True)
        

    # Add any columns they don't have
    columns_to_add = utah_col_set - state_col_set
    if len(columns_to_add) != 0:
        print("Cols to Add: ", columns_to_add)
        for column in columns_to_add:
            # Fill column with the utah mode
            states_X_test[column] = utah_modes[column].array[0]
        states_X_test = states_X_test[utah_cols]

    # Print the best params
    states_y_pred = model.predict(states_X_test)
    return f1_score(states_y_test, states_y_pred, average='macro', zero_division=1)

In [ ]:
# Get the folder with that contains the 
# dataset for each states accidents
state_filenames = os.listdir('./data/states')

us_f1score_dict = {}
utah_cols = X_train.columns
utah_col_set = set(utah_cols)
utah_modes = X_train.mode()

# Get the F1-score for each state using the Utah model
for state in tqdm(state_filenames):
    if state != "UT.csv":
        us_f1score_dict[state[:2]] = evaluate_state_f1score(state, utah_col_set, utah_cols, utah_modes)

In [54]:
# Create a pd.Series of the scores and find the average
ood_us = pd.Series(us_f1score_dict)
print('Average OOD score for United States (excluding Utah): ', ood_us.mean())

Average OOD score for United States (excluding Utah):  0.37354149326503233
